In [9]:
import pandas as pd
from keras.layers.embeddings import Embedding
from keras.layers import Flatten, Dense
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import numpy as np

In [3]:
np.random.seed(42)

In [4]:
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [5]:
# One hot encode for integer encoding
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[41, 4], [25, 4], [33, 4], [22, 4], [45], [16], [18, 4], [49, 25], [18, 4], [39, 34, 4, 7]]


In [6]:
# Pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[41  4  0  0]
 [25  4  0  0]
 [33  4  0  0]
 [22  4  0  0]
 [45  0  0  0]
 [16  0  0  0]
 [18  4  0  0]
 [49 25  0  0]
 [18  4  0  0]
 [39 34  4  7]]


In [7]:
# Define Sequential model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 80.000001


## Pre-trained embeddings

In [10]:
t = Tokenizer()
t.fit_on_texts(docs)

In [13]:
vocab_size = len(t.word_index) + 1

In [14]:
# Integer encode
encoded_docs = t.texts_to_sequences(docs)

In [15]:
encoded_docs

[[6, 2],
 [3, 1],
 [7, 4],
 [8, 1],
 [9],
 [10],
 [5, 4],
 [11, 3],
 [5, 1],
 [12, 13, 2, 14]]

In [16]:
max_pad_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_pad_length, padding='post')

In [18]:
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [19]:
# Load the GloVe embeddings

In [21]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('data/embeddings/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [33]:
# Load the embedding matrix of each unique word in the dataset
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [34]:
len(embedding_matrix)

15

In [36]:
embedding_matrix.shape

(15, 100)

In [37]:
vocab_size

15

In [40]:
# Create the new Embedding layer
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)

In [41]:
model = Sequential()
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [42]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None


In [43]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)

In [44]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)

In [45]:
loss, accuracy

(0.3977801203727722, 1.0)